# Лабораторна робота №9. Алгоритми на рядках.

Виконав - Огоновський Олександр

**Мета**:  засвоїти головні алгоритми на рядках засобами мови Python.


## 1. Реалізація прикладів

### 1.1. Наївний пошук шаблонів (The Naive String Matching Algorithm) 

In [15]:
def naive_match(p, t): 
    assert len(p) <= len(t)  # припускаємо, що довжина шаблону не більша за довжину тексту 
    occurrences = [] 
    for i in range(0, len(t)-len(p)+1): # для кожного вирівнювання шаблону на текст 
        match = True  # припускаємо, що є збіг, доки не доведено зворотнє 
        for j in range(0, len(p)):  # для кожної позиції шаблону 
            if t[i+j] != p[j]: 
                match = False  # принаймні один символ не збігається 
                break 
        if match: 
            occurrences.append(i) 
    return occurrences 

In [16]:
t = 'needleneedleneedle'
p = 'needle'
t[9 : 9 + len(p)]  # тобто t[9:15]

'dlenee'

In [17]:
naive_match('abc', 'abcabcabc') 

[0, 3, 6]

### 1.2. Z-функція 

In [18]:
def z_func(s): 
    # Ініціалізуємо масив Z, в якому будемо зберігати значення Z-функції 
    Z = [len(s)] + [0] * (len(s) - 1) # Встановлюємо довжину рядка s у першому елементі масиву Z, а решту заповнюємо нулями 
    
    assert len(s) > 1  # Перевіряємо, що довжина рядка s більше 1

    # Початкове порівняння s[1:] з префіксом s[0:]
    for i in range(1, len(s)):
        # Порівнюємо символи s[i] та s[i-1]. Якщо однакові — збільшуємо Z[1]
        if s[i] == s[i - 1]:
            Z[1] += 1
        else:
            break  # При першій невідповідності — зупиняємось

    # Встановлюємо межі правої частини (r) і лівої (l) збігу
    r, l = 0, 0
    if Z[1] > 0:
        l, r = 1, Z[1]

    # Обчислення Z-функції для кожного k від 2 до len(s) - 1
    for k in range(2, len(s)):
        assert Z[k] == 0  # Перевірка, що значення Z[k] спочатку дорівнює 0 

        if k > r:
            # Випадок 1: k за межами поточного діапазону [l, r] 
            for i in range(k, len(s)):
                # Порівнюємо символи s[i] та s[i-k]. Якщо вони співпадають, збільшуємо значення Z[k] 
                if s[i] == s[i - k]:
                    Z[k] += 1
                else:
                    break
            # Оновлюємо значення змінних r та l 
            l, r = k, k + Z[k] - 1
        else:
            # Випадок 2: k всередині поточного [l, r]
            # Обчислюємо довжину бета (nbeta) та значення Z-функції на попередньому кроці (Zkp)
            nbeta = r - k + 1
            Zkp = Z[k - l]
            if nbeta > Zkp:
                # Підвипадок 2а: повністю копіюємо
                Z[k] = Zkp
            else:
                # Підвипадок 2б: порівнюємо далі за межами r
                nmatch = 0
                for i in range(r + 1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                Z[k] = r - k + 1

    return Z

In [19]:
z_func('abracadabra') 
#  abracadabra (11) 
#     a        (1) 
#       a      (1) 
#         abra (4) 
#            a (1) 

[11, 0, 0, 1, 0, 1, 0, 4, 0, 0, 1]

### 1.3. Пошук підрядка в рядку 

In [20]:
def zMatch(p, t):
    # Формуємо розширений рядок s, де p – це шаблон, а t – текст, який шукаємо.
    s = p + "$" + t
    
    # Обчислюємо масив Z для рядка s.
    Z = z_func(s)
    
    # Ініціалізуємо список occurrences для зберігання позицій, де відбувається збіг з шаблоном.
    occurrences = []
    
    # Проходимо по всіх позиціях збігу шаблону з текстом у рядку s.
    # Починаємо з позиції, що відповідає кінцю шаблону (len(p) + 1) і до кінця рядка s.
    for i in range(len(p) + 1, len(s)):
        # Якщо значення Z на поточній позиції рівне довжині шаблону, то це означає, що відбувся збіг.
        if Z[i] == len(p):
            # Додаємо позицію, де відбувся збіг, до списку occurrences.
            # Позиція розраховується як i – (len(p) + 1), оскільки ми розглядаємо розширений рядок s,
            # де шаблон p розташований на початку, перед "$".
            occurrences.append(i - (len(p) + 1))
    
    # Повертаємо список знайдених збігів.
    return occurrences
t, p = "абабагаламага", "аб"
calculated_z = z_func("аб$абабагаламага")
print(calculated_z)

[16, 0, 0, 2, 0, 2, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]


### 1.4. Стиснення рядка 

In [21]:
def compress_with_z(s): 
    # Обчислюємо Z-вектор для рядка s 
    z_vec = z_func(s) 
     
    # Проходимо по елементам Z-вектора, починаючи з індексу 1 
    for i in range(1, len(s)): 
        # Перевіряємо, чи виконуються умови для оптимального стиснення 
        if (i + z_vec[i] == len(s)) and (z_vec[i] % i == 0): 
            # Якщо умови виконуються, повертаємо частину рядка s, яка буде стиснута 
            return s[:i] 
     
 # Якщо жодна пара (i, z_vec[i]) не відповідає умовам, повертаємо вихідний рядок s без змін 
    else: 
        return s 
 
s = "абырвалгабырвалгабырвалг" 
print(z_func(s)) 
print(compress_with_z(s)) 

[24, 0, 0, 0, 0, 1, 0, 0, 16, 0, 0, 0, 0, 1, 0, 0, 8, 0, 0, 0, 0, 1, 0, 0]
абырвалг


## 2. Робота алгоритмів з довілним рядком

In [14]:
t = "namelesslessnameless"
p = "nameless"


print("Z-функція для p + '$' + t:", z_func(p + "$" + t))
print("Z-Match:", zMatch(p, t))
print("Z-функція для стиснення:", z_func(t))

print("Naive Match:", naive_match(p, t))
print("Стиснення:", compress_with_z(t))

Z-функція для p + '$' + t: [29, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0]
Z-Match: [0, 12]
Z-функція для стиснення: [20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0]
Naive Match: [0, 12]
Стиснення: namelesslessnameless


## 3. Пояснення асимптотики наївного алгоритму пошуку підрядка. 

- n = len(t) — довжина тексту
- m = len(p) — довжина шаблону
У найгіршому випадку:  
Для кожного з n - m + 1 вирівнювання (ітерацій зовнішнього циклу) потрібно зробити до `m` порівнянь (внутрішній цикл).  
Усі порівняння символів — це O(m), а кількість позицій — $O(n - m + 1) \approx  O(n)$. 

Сумарна складність:
$O(n \cdot m)$

## 4. Обчислення асимптотичної складності алгоритму стиснення рядка за допомогою 𝑧-функції. 

1. Обчислення Z-функції:  
Вона рахується за O(n) — дуже швидко, де n — довжина рядка.  
2. Перевірка можливих варіантів шаблону:  
Ми просто дивимось на всі позиції 1 до n-1, тобто ще O(n).
3. Сумарна складність O(n) + O(n) = O(2n) = O(n).

## 5. Контрольні питання

### 1. Що таке «префікс-функція» у контексті алгоритмів на рядках, і як вона відрізняється від Z-функції? 

Префікс-функція — це масив, де в кожному елементі зберігається довжина найбільшого префікса, який одночасно є суфіксом для підрядка s[0:i], не включаючи повний рядок.

### 2. Що таке Z-функція у вищому контексті алгоритмів на рядках, і яка її роль у вирішенні задач?

Z-функція для рядка s — це масив Z, де Z[i] показує, яка максимальна довжина підрядка, що починається з позиції i і є префіксом всього рядка s.  
Для чого використовується Z-функція:
Пошук підрядка за допомогою шаблону.
Обчислення повторів, стиснення рядка
Аналіз тексту (пошук префіксів/суфіксів)

### 3. Які існують підходи до вирішення задачі «найдовший спільний підрядок» для двох рядків? 

Основні підходи:
- Динамічне програмування (DP) — класичний підхід:
Створюємо таблицю dp[i][j], де зберігається довжина спільного підрядка, що закінчується в s1[i] і s2[j].
Складність: O(n * m)
- Хешування (Rolling Hash):
Бінарний пошук по довжині + хеш
Складність: O(n log n)

### 4. Як можна застосувати алгоритми на рядках у задачах обробки природної мови або обробки текстів? 

Приклади застосування:

- Пошук слів/фраз у текстах — Z-функція, KMP
- Визначення схожості між текстами
- Автодоповнення / перевірка орфографії
- Аналіз частоти слів, дублювання — хеш-таблиці + пошук шаблонів
- Пошук плагіату — порівняння підрядків
- Стискання тексту — Z-функція
- Розпізнавання шаблонів (імейли, теги) —  пошук шаблонів